In [89]:
import tabula
import pandas as pd
import io
import math

In [90]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = None
filename = "Padrao_TISS_Componente_Organizacional__202012.pdf"

# Table 30

In [91]:
page = 79
top = 132.87
left = 134.41
bottom = top + 83.1
right = left + 212.2
col_boundary = 201.75

df = tabula.read_pdf(filename,
                     pages=page,
                     guess=False,
                     area=[top, left, bottom, right],
                     columns=[col_boundary])[0]
df

,Código,Descrição da categoria
0,1,Operadora
1,2,Prestador de serviço
2,3,Consumidor
3,4,Gestor
4,5,ANS


In [92]:
df.to_csv('table-30.csv', index=False)

# Table 32

In [93]:
page = 85
top = 147.08
left = 136.12
bottom = top + 54.73
right = left + 177.02
col_boundary = 169.6

df = tabula.read_pdf(filename,
                     pages=page,
                     guess=False,
                     area=[top, left, bottom, right],
                     columns=[col_boundary])[0]
df

,Código,Descrição da categoria
0,1,Alteração
1,2,Inclusão
2,3,Exclusão


In [94]:
df.to_csv('table-32.csv', index=False)

# Table 31

In [95]:
first_page = 79
last_page = 84 + 1  # need to add 1 since end index is exclusive
pages = list(range(first_page, last_page))
dfs = tabula.read_pdf(filename, pages=pages)[1:]

In [96]:
dfs[0]

,Unnamed: 0,Tabela de Categoria do Padrão TISS
0,Código,Descrição da categoria
1,1,Componente Organizacional
2,2,Componente de Conteúdo e Estrutura
3,3,Componente de Representação de Conceitos em Saúde


In [97]:
# Fix the first dataframe (which contains the header)
df = dfs[0]
headers = df.iloc[0]
dfs[0]  = pd.DataFrame(df.values[1:], columns=headers)
dfs[0]

,Código,Descrição da categoria
0,1,Componente Organizacional
1,2,Componente de Conteúdo e Estrutura
2,3,Componente de Representação de Conceitos em Saúde


In [98]:
# Set the headers in the remaining dataframes

def set_headers(df, headers):
    new_df = pd.read_csv(io.StringIO(df.to_csv(index=False)), header=None)
    return pd.DataFrame(new_df.values, columns=headers)

for i in range(1, len(dfs)):
    dfs[i] = set_headers(dfs[i], headers)

In [99]:
df = pd.concat(dfs, ignore_index=True)

In [100]:
df.columns

Index(['Código', 'Descrição da categoria'], dtype='object', name=0)

In [101]:
df

,Código,Descrição da categoria
0,1,Componente Organizacional
1,2,Componente de Conteúdo e Estrutura
2,3,Componente de Representação de Conceitos em Saúde
3,4,Componente de Comunicação
4,5,Componente de Segurança e Privacidade
5,18,"Terminologia de diárias, taxas e gases medicinais"
6,19,Terminologia de materiais e OPME
7,20,Terminologia de medicamentos
8,22,Terminologia de procedimentos e eventos em saúde
9,23,Terminologia de caráter do atendimento


In [102]:
'''
Tabula does not deal well with entries that take up more
than one line in the PDF.
We need to scan the df in order to find these entries
and fix them up.
'''

def fix_entries(df):
    new_df = pd.DataFrame(columns=df.columns)
    i = 1
    while i < df.shape[0]:
        row = df.iloc[i-1].copy()
        x = df.iloc[i]['Descrição da categoria']
        if isinstance(x, float) and math.isnan(x):
            suffix = df.iloc[i]['Código']
            row['Descrição da categoria'] += ' {}'.format(suffix)
            i += 1
        new_df = new_df.append(row, ignore_index=True)
        i += 1
        
    row = df.iloc[df.shape[0]-1].copy()  # copy last row
    new_df.append(row, ignore_index=True)
    return new_df

new_df = fix_entries(df)

In [103]:
# We can now export to CSV
new_df.to_csv('table-31.csv', index=False)